In [ ]:
!pip install tensorflow==2.4.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install keras==2.4.3 numpy==1.19.3 pillow==7.0.0 scipy==1.4.1 h5py==2.10.0 matplotlib==3.3.2 opencv-python keras-resnet==0.2.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install imageai --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


импорт библиотек

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn import preprocessing
import matplotlib.pyplot as plt

просмотрим содержимое csv файлов

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/moabd-l2-2022/train.csv')
train[:10]

,id,target_people
0,tr0001.jpg,0
1,tr0002.jpg,0
2,tr0003.jpg,1
3,tr0004.jpg,0
4,tr0005.jpg,0
5,tr0006.jpg,0
6,tr0007.jpg,1
7,tr0008.jpg,1
8,tr0009.jpg,1
9,tr0010.jpg,0


In [ ]:
valid = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/moabd-l2-2022/valid.csv')
valid

,id,target_people
0,v0001.jpg,1
1,v0002.jpg,0
2,v0003.jpg,0
3,v0004.jpg,0
4,v0005.jpg,0
...,...,...
1531,v1532.jpg,0
1532,v1533.jpg,1
1533,v1534.jpg,1
1534,v1535.jpg,1


In [ ]:
sam_sub = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/moabd-l2-2022/sample_submission.csv')
sam_sub.columns=['name','target_people']
sam_sub

,name,target_people
0,te0001.jpg,0.5
1,te0002.jpg,0.5
2,te0003.jpg,0.5
3,te0004.jpg,0.5
4,te0005.jpg,0.5
...,...,...
4518,te4519.jpg,0.5
4519,te4520.jpg,0.5
4520,te4521.jpg,0.5
4521,te4522.jpg,0.5


Начнем работу.
Для работы с изображениями будем использовать их абсолютные пути в файловой системе и поместим их в массив numpy.
glob находит все файлы с заданным названием - в данном случае *.jpg, то есть все джпеговские картинки - и помещает их название в список (обычный одномерный массив)

In [ ]:
import glob
import numpy as np
from PIL import Image

files = glob.glob("/content/drive/MyDrive/Colab Notebooks/moabd-l2-2022/test/*.jpg")
n = len(files)
X_data = []

for my_file in files:
    X_data.append(my_file)

X_data = np.array(X_data)

print('X_data shape:', X_data.shape)

X_data shape: (4523,)


#### Выбор модели и обоснование.
#### Для распознавания объектов на изображениях использовала библиотеку imageai - етсь в открытом доступе на гитхабе, можно почитать документацию https://imageai.readthedocs.io/en/latest/index.html
#### пригодился еще tensorflow 
#### импортируем библиотеку os для работы с путями и укажем важные для модели: exec_path - папка проекта, mpath - папка с файлами моделей (как я поняла, это методы обучения, может нейронные сети/перцептрон)


#### detector - объявление нового класса для поиска объектов;
#### мы создали экземпляр класса ObjectDetection, теперь мы можем вызывать различные функции из класса. Для распознавания людей будем использовать модель TinyYOLOv3, и, таким образом, функцию setModelTypeAsTinyYOLOv3() для загрузки модели.
#### теперь вызовем функцию setModelPath(), она примет строку, содержащую путь к модели.
#### затем функция loadModel() для загрузка модели (по указанному выше пути с помощью метода класса setModelPath) внутрь класса для поиска.

In [ ]:
import tensorflow as tf
from imageai.Detection import ObjectDetection
import os

exec_path = '/content/drive/MyDrive/Colab Notebooks/moabd-l2-2022/'
mpath = '/content/drive/MyDrive/Colab Notebooks/moabd-l2-2022/models/yolo-tiny.h5'

detector = ObjectDetection()
detector.setModelTypeAsTinyYOLOv3() 
detector.setModelPath(mpath)
detector.loadModel(detection_speed="fast")

#### в train_imgs - отсортированный массив названий картинок с абсолютным путем
#### .CustomObjects - "эта функция используется, когда вы хотите обнаружить только выбранное количество объектов. Он возвращает словарь объектов и их истинные или ложные значения."
#### потом проходимся по массиву картинок и вызываем функцию обнаружения (распознавания объектов) detectObjectsFromImage - в документации все подробно объяснено https://imageai.readthedocs.io/en/latest/detection/index.html
#### если хоть что-то распознали, то в массив найденных записываем название картинки (относительный путь) и вероятность нахождения человека

In [ ]:
test_imgs = np.sort(X_data)
custom = detector.CustomObjects(person=True)
detections = []

for te_img in test_imgs[:300]:
  print(te_img)
  returned_image, recognized = detector.detectObjectsFromImage(
                                                    custom_objects=custom, 
                                                    input_image=te_img, 
                                                    output_type="array", 
                                                    minimum_percentage_probability=10)
  if recognized:
    detections.append([
                        te_img[-10:], 
                        np.round(recognized[0]["percentage_probability"], 2)
                    ])
print(detections)

/content/drive/MyDrive/Colab Notebooks/moabd-l2-2022/test/te0001.jpg
/content/drive/MyDrive/Colab Notebooks/moabd-l2-2022/test/te0002.jpg
/content/drive/MyDrive/Colab Notebooks/moabd-l2-2022/test/te0003.jpg
/content/drive/MyDrive/Colab Notebooks/moabd-l2-2022/test/te0004.jpg
/content/drive/MyDrive/Colab Notebooks/moabd-l2-2022/test/te0005.jpg
/content/drive/MyDrive/Colab Notebooks/moabd-l2-2022/test/te0006.jpg
/content/drive/MyDrive/Colab Notebooks/moabd-l2-2022/test/te0007.jpg
/content/drive/MyDrive/Colab Notebooks/moabd-l2-2022/test/te0008.jpg
/content/drive/MyDrive/Colab Notebooks/moabd-l2-2022/test/te0009.jpg
/content/drive/MyDrive/Colab Notebooks/moabd-l2-2022/test/te0010.jpg
/content/drive/MyDrive/Colab Notebooks/moabd-l2-2022/test/te0011.jpg
/content/drive/MyDrive/Colab Notebooks/moabd-l2-2022/test/te0012.jpg
/content/drive/MyDrive/Colab Notebooks/moabd-l2-2022/test/te0013.jpg
/content/drive/MyDrive/Colab Notebooks/moabd-l2-2022/test/te0014.jpg
/content/drive/MyDrive/Colab Noteb

создадим датафрейм из найденных значений

In [ ]:
df = pd.DataFrame(detections,columns=['name','pred'])
df

,name,pred
0,te0001.jpg,12.34
1,te0003.jpg,25.25
2,te0008.jpg,77.58
3,te0011.jpg,11.87
4,te0014.jpg,10.71
...,...,...
114,te0285.jpg,17.34
115,te0292.jpg,46.98
116,te0295.jpg,16.83
117,te0296.jpg,22.52


объединим с датафреймом из sample_submission, чтобы значения вероятнойстей совпадали с названиями

In [ ]:
result = pd.merge(sam_sub, df, how='left')
result

,name,target_people,pred
0,te0001.jpg,0.5,12.34
1,te0002.jpg,0.5,NaN
2,te0003.jpg,0.5,25.25
3,te0004.jpg,0.5,NaN
4,te0005.jpg,0.5,NaN
...,...,...,...
4518,te4519.jpg,0.5,NaN
4519,te4520.jpg,0.5,NaN
4520,te4521.jpg,0.5,NaN
4521,te4522.jpg,0.5,NaN


заполним нули

In [ ]:
result = result.fillna(0)

сохраним в файл

In [ ]:
result = result.drop(['target_people'],axis=1)
result.to_csv('/content/drive/MyDrive/Colab Notebooks/moabd-l2-2022/sample_submission_res.csv', index=False)
result

,name,pred
0,te0001.jpg,12.34
1,te0002.jpg,0.00
2,te0003.jpg,25.25
3,te0004.jpg,0.00
4,te0005.jpg,0.00
...,...,...
4518,te4519.jpg,0.00
4519,te4520.jpg,0.00
4520,te4521.jpg,0.00
4521,te4522.jpg,0.00
